## Prepare

In [5]:
pip install open-clip-torch

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 70.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import torch
import torch.nn as nn
from os.path import expanduser  # pylint: disable=import-outside-toplevel
from urllib.request import urlretrieve  # pylint: disable=import-outside-toplevel
def get_aesthetic_model(clip_model="vit_l_14"):
    """load the aethetic model"""
    home = expanduser("~")
    cache_folder = home + "/.cache/emb_reader"
    path_to_model = cache_folder + "/sa_0_4_"+clip_model+"_linear.pth"
    if not os.path.exists(path_to_model):
        os.makedirs(cache_folder, exist_ok=True)
        url_model = (
            "https://github.com/LAION-AI/aesthetic-predictor/blob/main/sa_0_4_"+clip_model+"_linear.pth?raw=true"
        )
        urlretrieve(url_model, path_to_model)
    if clip_model == "vit_l_14":
        m = nn.Linear(768, 1)
    elif clip_model == "vit_b_32":
        m = nn.Linear(512, 1)
    else:
        raise ValueError()
    s = torch.load(path_to_model)
    m.load_state_dict(s)
    m.eval()
    return m

amodel= get_aesthetic_model(clip_model="vit_l_14")
amodel.eval()

import torch
from PIL import Image
import open_clip
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai')

/tmp/ipykernel_2562133/90848741.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  s = torch.load(path_to_model)


open_clip_model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

/home/luyaoton/.local/lib/python3.11/site-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


## Run

In [3]:

!wget https://thumbs.dreamstime.com/b/lovely-cat-as-domestic-animal-view-pictures-182393057.jpg

--2025-03-18 17:44:49--  https://thumbs.dreamstime.com/b/lovely-cat-as-domestic-animal-view-pictures-182393057.jpg
146.75.9.91humbs.dreamstime.com (thumbs.dreamstime.com)... 
Connecting to thumbs.dreamstime.com (thumbs.dreamstime.com)|146.75.9.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96578 (94K) [image/jpeg]
Saving to: ‘lovely-cat-as-domestic-animal-view-pictures-182393057.jpg’

lovely-cat-as-domes 100%[===================>]  94.31K  --.-KB/s    in 0.005s  

2025-03-18 17:44:49 (19.0 MB/s) - ‘lovely-cat-as-domestic-animal-view-pictures-182393057.jpg’ saved [96578/96578]



In [7]:
image = preprocess(Image.open("lovely-cat-as-domestic-animal-view-pictures-182393057.jpg")).unsqueeze(0)
with torch.no_grad():
    image_features = model.encode_image(image)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    prediction = amodel(image_features)
    print(prediction)

tensor([[4.4425]])


In [22]:
import torch
from PIL import Image
import open_clip
from pathlib import Path
import numpy as np

# Load models (assuming these are already initialized as in your notebook)
# If needed, uncomment and run these lines:
# model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai')
# amodel = get_aesthetic_model(clip_model="vit_l_14")
# amodel.eval()

def get_aesthetic_score(image_path):
    try:
        # Load and preprocess image
        image = preprocess(Image.open(image_path)).unsqueeze(0)
        
        with torch.no_grad():
            # Extract features and normalize
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            
            # Get aesthetic prediction
            prediction = amodel(image_features)
            return prediction.item()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None



In [13]:
# Section 1: Process Left/Right image sets
print("SECTION 1: Aesthetic Scores for Left/Right Image Sets")
print("=" * 60)
print("| Set | Left Image | Right Image |")
print("-" * 60)

for set_num in range(1, 6):  # Sets 1-5
    left_image = f"PhotoSet/set{set_num}L.jpg"
    right_image = f"PhotoSet/set{set_num}R.jpg"
    
    # Get scores
    left_score = get_aesthetic_score(left_image)
    right_score = get_aesthetic_score(right_image)
    
    # Print results
    print(f"| Set {set_num} | {left_score:.4f} | {right_score:.4f} |")
    
    # Optional: indicate which image has higher aesthetic score
    if left_score is not None and right_score is not None:
        if left_score > right_score:
            print(f"  → Left image scores higher by {left_score - right_score:.4f} points")
        elif right_score > left_score:
            print(f"  → Right image scores higher by {right_score - left_score:.4f} points")
        else:
            print("  → Both images have equal scores")
    
    print("-" * 60)


SECTION 1: Aesthetic Scores for Left/Right Image Sets
| Set | Left Image | Right Image |
------------------------------------------------------------
| Set 1 | 4.7169 | 5.6019 |
  → Right image scores higher by 0.8850 points
------------------------------------------------------------
| Set 2 | 4.1938 | 5.8174 |
  → Right image scores higher by 1.6236 points
------------------------------------------------------------
| Set 3 | 5.7389 | 4.1136 |
  → Left image scores higher by 1.6253 points
------------------------------------------------------------
| Set 4 | 6.8118 | 5.6914 |
  → Left image scores higher by 1.1204 points
------------------------------------------------------------
| Set 5 | 5.2023 | 4.4697 |
  → Left image scores higher by 0.7325 points
------------------------------------------------------------


In [ ]:
# Section 2: Process Multi-image sets
print("\n\nSECTION 2: Aesthetic Scores for Multi-Image Sets")
print("=" * 80)

# Process multi-image sets
# Format: multiSet{set_num}_{total_images}_{image_index}.jpg
# Example: multiSet1_5_1.jpg means set 1, 5 total images, image 1

# Define the multi-sets to process
multi_sets = [
    {"set_num": 1, "total_images": 5}
    # Add more multi-sets here if needed
]

for multi_set in multi_sets:
    set_num = multi_set["set_num"]
    total_images = multi_set["total_images"]
    
    print(f"Multi-Set {set_num} (Total Images: {total_images})")
    print("-" * 80)
    print("| Image | Filename | Aesthetic Score |")
    print("-" * 80)
    
    scores = []
    for img_idx in range(1, total_images + 1):
        img_filename = f"PhotoSet/multiSet{set_num}_{total_images}_{img_idx}.jpg"
        
        # Get score for this image
        score = get_aesthetic_score(img_filename)
        scores.append(score)
        
        # Print individual image score
        print(f"| {img_idx} | {img_filename} | {score:.4f} |")
    
    # Print summary statistics
    if scores and all(s is not None for s in scores):
        scores = np.array(scores)
        print("-" * 80)
        print(f"Summary Statistics for Multi-Set {set_num}:")
        print(f"  → Average Score: {np.mean(scores):.4f}")
        print(f"  → Median Score: {np.median(scores):.4f}")
        print(f"  → Min Score: {np.min(scores):.4f} (Image {np.argmin(scores) + 1})")
        print(f"  → Max Score: {np.max(scores):.4f} (Image {np.argmax(scores) + 1})")
        print(f"  → Standard Deviation: {np.std(scores):.4f}")
    
    print("=" * 80)
    print()



SECTION 2: Aesthetic Scores for Multi-Image Sets
Multi-Set 1 (Total Images: 5)
--------------------------------------------------------------------------------
| Image | Filename | Aesthetic Score |
--------------------------------------------------------------------------------
